# 1. Import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import gc
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import root_mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost.callback import EarlyStopping
from lightgbm import LGBMRegressor, Dataset
from catboost import Pool, CatBoostRegressor
from prettytable import PrettyTable
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer, FunctionTransformer
from sklearn.model_selection import cross_val_score, KFold
import optuna
from sklearn.metrics import make_scorer
from sklearn.ensemble import VotingRegressor
from sklearn.compose import TransformedTargetRegressor
import warnings
warnings.filterwarnings("ignore")

# 2. Load data

## 2.1 Load train/data datasets

Original data was downloaded from [here](https://archive.ics.uci.edu/dataset/1/abalone)

In [ ]:
# Import abalone data from Kaggle
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

# Import original data
original_names = pd.read_csv('data/abalone.names', names=['names'])
original = pd.read_csv('data/abalone.data', names=original_names['names'])

# Copy train and test datasets
train_copy = train.copy()
test_copy = test.copy()

# Tag original dataset
original["original"] = 1
train["original"] = 0
test["original"] = 0

# Drop 'id' columns
train.drop(columns=['id'], inplace=True)
test.drop(columns=['id'], inplace=True)

# Rename train and test column names to match names of the original dataset
train = train.rename(columns={'Whole weight':'Whole_weight','Whole weight.1':'Shucked_weight', 'Whole weight.2':'Viscera_weight', 'Shell weight':'Shell_weight'})
test = test.rename(columns={'Whole weight':'Whole_weight','Whole weight.1':'Shucked_weight', 'Whole weight.2':'Viscera_weight', 'Shell weight':'Shell_weight'})

# Concatenate train and original datasets
train = pd.concat([train, original], axis='rows')
train.reset_index(inplace=True, drop=True)

# Set device
device='gpu'

# Set target
target='Rings'

train.head()

## 2.2 Check for missing values

In [ ]:
table = PrettyTable()

table.field_names = ['Column Name', 'Data Type', 'Train Missing %', 'Test Missing %']
for column in train.columns:
    data_type = str(train[column].dtype)
    non_null_count_train = 100-train[column].count()/train.shape[0]*100
    if column != target:
        non_null_count_test = 100-test[column].count()/test.shape[0]*100
    else:
        non_null_count_test = "NA"
    table.add_row([column, data_type, non_null_count_train,non_null_count_test])
print(table)

# 3. Exploratory data analysis

# 3.1 Target analysis

In [ ]:
def histogram_density(class_0, class_1, title):
    # Calculate mean and median of targets
    mean_0 = np.mean(class_0)
    mean_1 = np.mean(class_1)
    median_0 = np.median(class_0)
    median_1 = np.median(class_1)

    # Draw histogram and density plots
    fig, ax = plt.subplots(figsize=(12, 6))

    # Create histograms
    ax.hist(class_0, bins=20, density=True, alpha=0.5, label='Original=0 Histogram')
    ax.hist(class_1, bins=20, density=True, alpha=0.5, label='Original=1 Histogram')

    # Get x-axis values for density plot (class 0 and class 1)
    x_values_0 = np.linspace(class_0.min(), class_0.max(), len(class_0))
    x_values_1 = np.linspace(class_1.min(), class_1.max(), len(class_0))

    # Get density values of class 0 and class 1
    density_values_0 = (1 / (np.sqrt(2 * np.pi) * np.std(class_0))) * np.exp(-0.5 * ((x_values_0 - mean_0) / np.std(class_0))**2)
    density_values_1 = (1 / (np.sqrt(2 * np.pi) * np.std(class_1))) * np.exp(-0.5 * ((x_values_1 - mean_1) / np.std(class_1))**2)

    # Draw density plots
    ax.plot(x_values_0, density_values_0, color='red', label='Original=0 Density')
    ax.plot(x_values_1, density_values_1, color='green', label='Original=1 Density')

    # Draw mean plots
    ax.axvline(mean_0, color='blue', linestyle='dashed', linewidth=2, label='Mean (Original=0)')
    ax.axvline(mean_1, color='blue', linestyle='dashed', linewidth=2, label='Mean (Original=1)')

    # Draw median plots
    ax.axvline(median_0, color='green', linestyle='dashed', linewidth=2, label='Median (Original=0)')
    ax.axvline(median_1, color='red', linestyle='dashed', linewidth=2, label='Median (Original=1)')

    # Set labels and a title
    ax.set_xlabel(target)
    ax.set_ylabel('Frequency / Density')
    ax.set_title(title)

    # Set x-axis view limits
    x_min = min(min(class_0), min(class_1))
    x_max = max(max(class_0), max(class_1))
    ax.set_xlim([x_min, x_max])

    # Create a legend
    ax.legend(bbox_to_anchor=(1,1),fancybox=False,shadow=False, loc='upper left')

    # Show plot
    plt.tight_layout()
    plt.show()

### 3.1.1 Histogram and density plots

In [ ]:
# Get target columns for train and original datasets
class_0 = train[train['original'] == 0][target]
class_1 = train[train['original'] == 1][target]

# Draw plot
histogram_density(class_0, class_1, 'Histograms and Density Plots')

### 3.1.2 Log transformed histogram and density plots

In [ ]:
# Get log transformed target columns for train and original datasets
class_0 = np.log1p(train[train['original'] == 0][target])
class_1 = np.log1p(train[train['original'] == 1][target])

# Draw plot
histogram_density(class_0, class_1, 'Log Transformed Histograms and Density Plots')

## 3.2 Train & test data distributions

In [ ]:
# Plot only numeric parameters with more than 2 unique elements (exclude target column)
cont_cols=[f for f in train.columns if train[f].dtype in [float,int] and train[f].nunique() > 2 and f not in [target]]

# Calculate number of rows needed for the subplots
num_rows = len(cont_cols)
while num_rows % 3 != 0: num_rows += 1
num_rows //= 3

# Create subplots for each continuous column
fig, axs = plt.subplots(num_rows, 3, figsize=(15, num_rows*5))

# Loop through each continuous column and plot the histograms
for i, col in enumerate(cont_cols):
    # Determine range of values to plot
    max_val = max(train[col].max(), test[col].max(), original[col].max())
    min_val = min(train[col].min(), test[col].min(), original[col].min())
    range_val = max_val - min_val

    # Determine the bin size and number of bins
    bin_size = range_val / 20
    num_bins_train = round(range_val / bin_size)
    num_bins_test = round(range_val / bin_size)
    num_bins_original = round(range_val / bin_size)

    # Calculate the subplot position
    row = i // 3
    col_pos = i % 3

    # Plot the histograms
    sns.histplot(train[col], ax=axs[row][col_pos], color='orange', kde=True, label='Train', bins=num_bins_train)
    sns.histplot(test[col], ax=axs[row][col_pos], color='green', kde=True, label='Test', bins=num_bins_test)
    sns.histplot(original[col], ax=axs[row][col_pos], color='blue', kde=True, label='Original', bins=num_bins_original)
    axs[row][col_pos].set_title(col)
    axs[row][col_pos].set_xlabel('Value')
    axs[row][col_pos].set_ylabel('Frequency')
    axs[row][col_pos].legend()

# Remove any empty subplots
if len(cont_cols) % 3 != 0:
    for col_pos in range(len(cont_cols) %3, 3):
        axs[-1][col_pos].remove()

# Show the plot
plt.tight_layout()
plt.show()

## 3.3 Sex & Numerical features

In [ ]:
sns.pairplot(data=original, vars=cont_cols+[target], hue='Sex')
plt.show()

## 3.4 Rings vs Sex

In [ ]:
plt.subplots(figsize=(16, 5))
sns.violinplot(x = 'Sex', y = col, data = train, palette = 'pastel')
plt.title('Rings Distribution by Sex', fontsize=14)
plt.xlabel('Sex', fontsize=12)
plt.ylabel('Rings', fontsize=12)
sns.despine()
fig.tight_layout()
plt.show()

## 3.5. Correlation matrix

In [ ]:
# Correlation of continuous data in test set
features = [f for f in test.columns if f != 'Sex']
corr = train[features].corr()
plt.figure(figsize = (8, 6), dpi = 300)
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr, mask = mask, cmap = 'magma', annot = True, annot_kws = {'size' : 6})
plt.title('Features Correlation Matrix\n', fontsize = 15, weight = 'bold')
plt.show()

# 4. Feature Engineering

## 4.1 Basic functions

In [2]:
def MSE(y_true, y_pred):
    return np.square(np.subtract(y_true, y_pred)).mean()

def RMSE(y_true, y_pred):
    return np.sqrt(np.square(np.subtract(y_true, y_pred)).mean())

def RMSLE(y_true, y_pred):
    return np.sqrt(np.square(np.subtract(np.log(1 + y_pred), np.log(1 + y_true))).mean())

def min_max_scaler(train, test, column):
    scaler = MinMaxScaler()
    max_val = max(train[column].max(), test[column].max())
    min_val = min(train[column].min(), test[column].min())
    train[column] = (train[column] - min_val) / (max_val - min_val)
    test[column] = (test[column] - min_val) / (max_val - min_val)
    return train, test

#global y_unique
#y_unique=train["Rings"].unique()

def nearest(y_predicted):
    
    y_original=y_unique
    modified_prediction = np.zeros_like(y_predicted)

    for i, y_pred in enumerate(y_predicted):
        nearest_value = min(y_original, key=lambda x: abs(x - y_pred))
        modified_prediction[i] = nearest_value

    return modified_prediction

## 4.2 New Features

1. ***Top Surface Area:*** Length X Diameter
2. ***Water Loss:*** During the experiment of data collection, it is possible to have some water loss after dissecting the crab to measure different weights
3. ***Abalone Density:*** Measure of body density
4. ***BMI:*** Body Mass index
5. ***Measurement ratios:*** You can also calculate ratios like length/height, length/diameter.
6. ***Incorrect Weights:*** I have noticed that there are sub weight columns greater than the total weight of the Abalone. Part of the body cannot have more weight that the whole body

In [ ]:
def new_features(data):
    df=data.copy()
    
    # Clean the weights by capping the over weights with total body weights
    df['Shell_weight']=np.where(df['Shell_weight']>df['Whole_weight'],df['Whole_weight'],df['Shell_weight'])
    df['Viscera_weight']=np.where(df['Viscera_weight']>df['Whole_weight'],df['Whole_weight'],df['Viscera_weight'])
    df['Shucked_weight']=np.where(df['Shucked_weight']>df['Whole_weight'],df['Whole_weight'],df['Shucked_weight'])
    
    # Abalone Surface area
    df["surface_area"]=df["Length"]*df["Diameter"]
    df['total_area']=2*(df["surface_area"]+df["Height"]*df["Diameter"]+df["Length"]*df["Height"])
    
    # Abalone density approx
    df['approx_density']=df['Whole_weight']/(df['surface_area']*df['Height']+1e-5)
    
    # Abalone BMI
    df['bmi']=df['Whole_weight']/(df['Height']**2+1e-5)
    
    # Measurement derived
    df["length_dia_ratio"]=df['Length']/(df['Diameter']+1e-5)
    df["length_height_ratio"]=df['Length']/(df['Height']+1e-5)
    df['shell_shuck_ratio']=df["Shell_weight"]/(df["Shucked_weight"]+1e-5)
    df['shell_viscera_ratio']=df['Shell_weight']/(df['Viscera_weight']+1e-5)
    
    df['viscera_tot_ratio']=df['Viscera_weight']/(df['Whole_weight']  +1e-5)
    df['shell_tot_ratio']=df['Shell_weight']/(df['Whole_weight']    +1e-5)
    df['shuck_tot_ratio']=df['Shucked_weight']/(df['Whole_weight']   +1e-5)
    df['shell_body_ratio']=df['Shell_weight']/(df['Shell_weight']+df['Whole_weight']+1e-5)
    df['flesh_ratio']=df['Shucked_weight']/(df['Whole_weight']+df['Shucked_weight']+1e-5)
    
    df['inv_viscera_tot']= df['Whole_weight'] / (df['Viscera_weight']+1e-5)
    df['inv_shell_tot']= df['Whole_weight'] /( df['Shell_weight']+1e-5)
    df['inv_shuck_tot']= df['Whole_weight'] / (df['Shucked_weight']+1e-5)
    
    
    # Water Loss during experiment
    df["water_loss"]=df["Whole_weight"]-df["Shucked_weight"]-df['Viscera_weight']-df['Shell_weight']
    df["water_loss"]=np.where(df["water_loss"]<0,min(df["Shucked_weight"].min(),df["Viscera_weight"].min(),df["Shell_weight"].min()),df["water_loss"])
    return df

train = new_features(train)
test = new_features(test)
original = new_features(original)

## 4.3 Data Transformation

In [ ]:
# Separate labels from data
y = train["Rings"]
train = train.drop(['original', 'Rings'], axis=1)
test = test.drop(['original'], axis=1)

# Get numerical columns
columns = [col for col in train.columns if train[col].dtype not in ["object"]]

In [ ]:
global unimportant_features
global overall_best_score
global overall_best_col
unimportant_features=[]
overall_best_score=1e5
overall_best_col='none'
def transform_column(train, test, columns, target):
    # Make a copy of an original dataframe
    train_copy = train.copy()
    test_copy = test.copy()
    # Get feature and score variables
    global unimportant_features
    global overall_best_score
    global overall_best_col
    # Instantiate a table with field names
    table = PrettyTable()
    table.field_names = ['Feature', 'Original RMSLE', 'Transformation', 'Tranformed RMSLE']
    # Define parameters
    epsilon = 1e-5
    power_0_25 = lambda x: np.power(x + 1 - np.min(x), 0.25)
    power_2 = lambda x: np.power(x + 1 - np.min(x), 2)
    # Define transformers
    transformer_box_cox = PowerTransformer(method = 'box-cox')
    transformer_yeo_johnson = PowerTransformer(method = 'yeo-johnson')
    transformer_power_0_25 = FunctionTransformer(power_0_25)
    transformer_power_2 = FunctionTransformer(power_2)
    # Append transformed columns to the datafame
    for col in columns:
        # Scale train and test data
        train, test = min_max_scaler(train, test, col)
        # Log transformation
        train_copy['log_'+col] = np.log1p(train_copy[col])
        test_copy['log_'+col] = np.log1p(test_copy[col])
        # Square root transformation
        train_copy['sqrt_'+col] = np.sqrt(train_copy[col])
        test_copy['sqrt_'+col] = np.sqrt(test_copy[col])
        # Box Cox transformation
        train_copy['box_cox_'+col] = transformer_box_cox.fit_transform(train_copy[[col]] + epsilon)
        test_copy['box_cox_'+col] = transformer_box_cox.fit_transform(test_copy[[col]] + epsilon)
        # Yeo Johnson transformation
        train_copy['yeo_johnson_'+col] = transformer_yeo_johnson.fit_transform(train_copy[[col]])
        test_copy['yeo_johnson_'+col] = transformer_yeo_johnson.fit_transform(test_copy[[col]])
        # **0.25 transformation
        train_copy['power_0_25_'+col] = transformer_power_0_25.fit_transform(train_copy[[col]])
        test_copy['power_0_25_'+col] = transformer_power_0_25.fit_transform(test_copy[[col]])
        # **2 transformation
        train_copy['power_0_2_'+col] = transformer_power_2.fit_transform(train_copy[[col]])
        test_copy['power_0_2_'+col] = transformer_power_2.fit_transform(test_copy[[col]])
        # Log to power transformation
        train_copy['log_sqrt_'+col] = np.log1p(np.sqrt(train_copy[col]))
        test_copy['log_sqrt_'+col] = np.log1p(np.sqrt(test_copy[col]))
        # Get transformed column names
        transformed_columns = [col, "log_"+col, "sqrt_"+col, "box_cox_"+col, "yeo_johnson_"+col,  "power_0_25_"+col , "power_0_2_"+col,"log_sqrt_"+col]
        # TruncatedSVD
        pca = TruncatedSVD(n_components=1)
        x_pca_train = pca.fit_transform(train_copy[transformed_columns])
        x_pca_test = pca.transform(test_copy[transformed_columns])
        x_pca_train = pd.DataFrame(x_pca_train, columns=[col+"_pca_comb"])
        x_pca_test = pd.DataFrame(x_pca_test, columns=[col+"_pca_comb"])
        transformed_columns.append(col+"_pca_comb")
        
        test_copy = test_copy.reset_index(drop=True)
        
        train_copy = pd.concat([train_copy, x_pca_train], axis='columns')
        test_copy = pd.concat([test_copy, x_pca_test], axis='columns')
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        # Evaluate performance of each data transformation
        rmse_scores = []
        for t_col in transformed_columns:
            # Get array of train and test columns
            X = train_copy[[t_col]].values
            y = target.values
            # Save rmse score for every fold
            rmses = []
            for train_idx, val_idx in kf.split(X, y):
                X_train, y_train = X[train_idx], y[train_idx]
                x_val, y_val = X[val_idx], y[val_idx]
                model = LinearRegression()
                model.fit(X_train, np.log1p(y_train))
                y_pred = nearest(np.expm1(model.predict(x_val)))
                rmses.append(RMSE(np.log1p(y_val),np.log1p(y_pred)))
            rmse_scores.append((t_col, np.mean(rmses)))

            if overall_best_score > np.mean(rmses):
                overall_best_score = np.mean(rmses)
                overall_best_col = t_col
            if t_col == col:
                orig_rmse = np.mean(rmses)
            
        best_col, best_rmse = sorted(rmse_scores, key=lambda x: x[1], reverse=False)[0]
        cols_to_drop = [col for col in transformed_columns if col != best_col]
        final_selection = [col for col in transformed_columns if col not in cols_to_drop]
        if cols_to_drop:
            unimportant_features = unimportant_features+cols_to_drop
        table.add_row([col, orig_rmse, best_col, best_rmse])
    print(table)
    print(f"Overall best CV RMSLE score: {overall_best_score}")
    return train_copy, test_copy, transformed_columns

In [ ]:
%%time
train_copy, test_copy, transformed_columns = transform_column(train, test, columns, y)

In [ ]:
# Create backup of data
train_bak = train
test_bak = test

# Copy transformed data
train = train_copy
test = test_copy

## 4.4 Numerical Clustering

In [ ]:
train["Rings"] = y

In [ ]:
%%time

table = PrettyTable()
table.field_names = ['Cluster WOE Feature', 'RMSLE (CV-TRAIN)']
for col in columns:
    sub_set=[f for f in unimportant_features if col in f]
    #print(sub_set)
    temp_train=train[sub_set]
    temp_test=test[sub_set]
    sc=StandardScaler()
    temp_train=sc.fit_transform(temp_train)
    temp_test=sc.transform(temp_test)
    model = KMeans()

    # print(ideal_clusters)
    kmeans = KMeans(n_clusters=5)
    kmeans.fit(np.array(temp_train))
    labels_train = kmeans.labels_

    train[col+"_unimp_cluster_WOE"] = labels_train
    test[col+"_unimp_cluster_WOE"] = kmeans.predict(np.array(temp_test))
    
    kf=KFold(n_splits=5, shuffle=True, random_state=42)
    
    X=train[[col+"_unimp_cluster_WOE"]].values
    y=train[target].values

    best_rmse=[]
    for train_idx, val_idx in kf.split(X,y):
        X_train,y_train=X[train_idx],y[train_idx]
        x_val,y_val=X[val_idx],y[val_idx]
        model=LinearRegression()
        model.fit(X_train,np.log1p(y_train))
        y_pred=nearest(np.expm1(model.predict(x_val)))
        best_rmse.append(RMSE(np.log1p(y_val),np.log1p(y_pred)))
        
    table.add_row([col+"_unimp_cluster_WOE",np.mean(best_rmse)])
    if overall_best_score<np.mean(best_rmse):
            overall_best_score=np.mean(best_rmse)
            overall_best_col=col+"_unimp_cluster_WOE"
    
print(table)

In [ ]:
train = train_copy
test = test_copy

## 4.4 Encode categorical data

Encode categorical data using `One-Hot` encoding method

In [ ]:
cat_cols = [f for f in test.columns if (train[f].dtype != 'O' and train[f].nunique()<2000 and train[f].nunique()>2 and "WOE" not in f) or (train[f].dtype == 'O') ]
#print(train[cat_cols].nunique())

In [ ]:
train_bak = train
test_bak = test

In [ ]:
train = train_bak
test = test_bak

In [ ]:
%%time
def nearest_val(target):
    return min(common, key=lambda x: abs(x - target), default="EMPTY")

cat_cols_updated=['Sex']
for col in cat_cols:
    if train[col].dtype!="O":
        train[f"{col}_cat"]=train[col]
        test[f"{col}_cat"]=test[col]
        cat_cols_updated.append(f"{col}_cat")
        uncommon=list((set(test[col].unique())| set(train[col].unique()))-(set(test[col].unique())& set(train[col].unique())))
        if uncommon:
            common=list(set(test[col].unique())& set(train[col].unique()))
            train[f"{col}_cat"]=train[col].apply(nearest_val)
            test[f"{col}_cat"]=test[col].apply(nearest_val)
print(train[cat_cols_updated].nunique())

In [ ]:
len(train.columns), len(test.columns)

In [ ]:
train_bak = train
test_bak = test

In [ ]:
y = train['Rings']

In [ ]:
train = train_bak
test = test_bak

In [ ]:
train = train.replace('EMPTY', np.nan)
train = train.dropna(axis=1)

In [ ]:
test = test.replace('EMPTY', np.nan)
test = test.dropna(axis=1)

In [ ]:
tmp = [f for f in cat_cols_updated if f in train.columns]
cat_cols_updated = tmp

In [ ]:
cat_cols_updated

In [ ]:
global overall_best_score
# overall_best_score = 0
def OHE(train_df,test_df,cols,target):
    '''
    Function for one hot encoding, it first combines the data so that no category is missed and
    the category with least frequency can be dropped because of redundancy
    '''
    combined = pd.concat([train_df, test_df], axis=0)
    for col in cols:
        one_hot = pd.get_dummies(combined[col])
        counts = combined[col].value_counts()
        min_count_category = counts.idxmin()
        one_hot = one_hot.drop(min_count_category, axis=1)
        one_hot.columns=[str(f)+col+"_OHE" for f in one_hot.columns]
        combined = pd.concat([combined, one_hot], axis="columns")
        combined = combined.loc[:, ~combined.columns.duplicated()]
    
    # split back to train and test dataframes
    train_ohe = combined[:len(train_df)]
    test_ohe = combined[len(train_df):]
    test_ohe.reset_index(inplace=True,drop=True)
    test_ohe.drop(columns=[target],inplace=True)
    return train_ohe, test_ohe

def high_freq_ohe(train, test, extra_cols, target, n_limit=50):
    '''
    If you wish to apply one hot encoding on a feature with so many unique values, then this can be applied, 
    where it takes a maximum of n categories and drops the rest of them treating as rare categories
    '''
    train_copy=train.copy()
    test_copy=test.copy()
    ohe_cols=[]
    for col in extra_cols:
        dict1=train_copy[col].value_counts().to_dict()
        ordered=dict(sorted(dict1.items(), key=lambda x: x[1], reverse=True))
        rare_keys=list([*ordered.keys()][n_limit:])
#         ext_keys=[f[0] for f in ordered.items() if f[1]<50]
        rare_key_map=dict(zip(rare_keys, np.full(len(rare_keys),9999)))
        
        train_copy[col]=train_copy[col].replace(rare_key_map)
        test_copy[col]=test_copy[col].replace(rare_key_map)
    train_copy, test_copy = OHE(train_copy, test_copy, extra_cols, target)
    drop_cols=[f for f in train_copy.columns if "9999" in f or train_copy[f].nunique()==1]
    train_copy=train_copy.drop(columns=drop_cols)
    test_copy=test_copy.drop(columns=drop_cols)
    
    return train_copy, test_copy

def cat_encoding(train, test,cat_cols_updated, target):
    global overall_best_score
    global overall_best_col
    table = PrettyTable()
    table.field_names = ['Feature', 'Encoded Features', 'RMSLE Score']
    train_copy=train.copy()
    test_copy=test.copy()
    train_dum = train.copy()
    for feature in cat_cols_updated:
#         print(feature)
#         cat_labels = train_dum.groupby([feature])[target].mean().sort_values().index
#         cat_labels2 = {k: i for i, k in enumerate(cat_labels, 0)}
#         train_copy[feature + "_target"] = train[feature].map(cat_labels2)
#         test_copy[feature + "_target"] = test[feature].map(cat_labels2)

        dic = train[feature].value_counts().to_dict()
        train_copy[feature + "_count"] =train[feature].map(dic)
        test_copy[feature + "_count"] = test[feature].map(dic)

        dic2=train[feature].value_counts().to_dict()
#         list1=np.arange(len(dic2.values()),0,-1) # Higher rank for high count
        list1=np.arange(len(dic2.values())) # Higher rank for low count
        dic3=dict(zip(list(dic2.keys()),list1))
        
        train_copy[feature+"_count_label"]=train[feature].replace(dic3).astype(float)
        test_copy[feature+"_count_label"]=test[feature].replace(dic3).astype(float)

        temp_cols = [ feature + "_count", feature + "_count_label"]#,feature + "_target"
        #print(train_copy['box_cox_Length_cat'].astype(str))
        train_copy[feature]=train_copy[feature].astype(str)+"_"+feature
        test_copy[feature]=test_copy[feature].astype(str)+"_"+feature
        
        if train_copy[feature].nunique()<=15:
            train_copy[feature]=train_copy[feature].astype(str)+"_"+feature
            test_copy[feature]=test_copy[feature].astype(str)+"_"+feature
            train_copy, test_copy = OHE(train_copy, test_copy, [feature], target)
            
        else:
            train_copy,test_copy=high_freq_ohe(train_copy,test_copy,[feature], target, n_limit=15)
            
        train_copy=train_copy.drop(columns=[feature])
        #test_copy=test_copy.drop(columns=[feature])

        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        rmse_scores = []

        for f in temp_cols:
            X = train_copy[[f]].values
            y = train_copy[target].astype(int).values

            rmses = []
            for train_idx, val_idx in kf.split(X, y):
                X_train, y_train = X[train_idx], y[train_idx]
                x_val, y_val = X[val_idx], y[val_idx]
                model=LinearRegression()
                model.fit(X_train,np.log1p(y_train))
                y_pred=nearest(np.expm1(model.predict(x_val)))
                rmses.append(RMSE(np.log1p(y_val),np.log1p(y_pred)))
            rmse_scores.append((f,np.mean(rmses)))
            if overall_best_score > np.mean(rmses):
                overall_best_score = np.mean(rmses)
                overall_best_col = f
        best_col, best_auc = sorted(rmse_scores, key=lambda x: x[1], reverse=False)[0]

        corr = train_copy[temp_cols].corr(method='pearson')
        corr_with_best_col = corr[best_col]
        cols_to_drop = [f for f in temp_cols if corr_with_best_col[f] > 0.5 and f != best_col]
        final_selection = [f for f in temp_cols if f not in cols_to_drop]
        if cols_to_drop:
            train_copy = train_copy.drop(columns=cols_to_drop)
            test_copy = test_copy.drop(columns=cols_to_drop)

        table.add_row([feature, best_col, best_auc])
        #print(feature)
    print(table)
    print("overall best CV score: ", overall_best_score)
    return train_copy, test_copy

train, test= cat_encoding(train, test,cat_cols_updated, target)

In [ ]:
len(train.columns), len(test.columns)

In [ ]:
train_bak = train
test_bak = test

In [ ]:
train = train_bak
test = test_bak

In [ ]:
obj_cols = [col for col in train.columns if train[col].dtype == 'O']
obj_cols

In [ ]:
train=train.drop(columns=obj_cols)
test=test.drop(columns=obj_cols)

In [ ]:
first_drop=[ f for f in unimportant_features if f in train.columns]
train=train.drop(columns=first_drop)
test=test.drop(columns=first_drop)

In [ ]:
final_drop_list=[]

table = PrettyTable()
table.field_names = ['Original', 'Final Transformation', "RMSLE(CV)- Regression"]
dt_params={'criterion': 'absolute_error'}
threshold=0.85
# It is possible that multiple parent features share same child features, so store selected features to avoid selecting the same feature again
best_cols=[]

for col in cat_cols:
    sub_set=[f for f in train.columns if col in f and train[f].nunique()>100]
    print(sub_set)
    if len(sub_set)>2:
        correlated_features = []

        for i, feature in enumerate(sub_set):
            # Check correlation with all remaining features
            for j in range(i+1, len(sub_set)):
                correlation = np.abs(train[feature].corr(train[sub_set[j]]))
                # If correlation is greater than threshold, add to list of highly correlated features
                if correlation > threshold:
                    correlated_features.append(sub_set[j])

        # Remove duplicate features from the list
        correlated_features = list(set(correlated_features))
        print(correlated_features)
        if len(correlated_features)>=2:
            print(len(correlated_features))
            temp_train=train[correlated_features]
            temp_test=test[correlated_features]
            #Scale before applying PCA
            sc=StandardScaler()
            temp_train=sc.fit_transform(temp_train)
            temp_test=sc.transform(temp_test)

            # Initiate PCA
            pca=TruncatedSVD(n_components=1)
            x_pca_train=pca.fit_transform(temp_train)
            x_pca_test=pca.transform(temp_test)
            x_pca_train=pd.DataFrame(x_pca_train, columns=[col+"_pca_comb_final"])
            x_pca_test=pd.DataFrame(x_pca_test, columns=[col+"_pca_comb_final"])
            train=pd.concat([train,x_pca_train],axis='columns')
            test=pd.concat([test,x_pca_test],axis='columns')

            # Clustering
            model = KMeans()
            kmeans = KMeans(n_clusters=28)
            kmeans.fit(np.array(temp_train))
            labels_train = kmeans.labels_

            train[col+'_final_cluster'] = labels_train
            test[col+'_final_cluster'] = kmeans.predict(np.array(temp_test))

            cat_labels=cat_labels=train.groupby([col+"_final_cluster"])[target].mean()
            cat_labels2=cat_labels.to_dict()
            train[col+"_final_cluster"]=train[col+"_final_cluster"].map(cat_labels2)
            test[col+"_final_cluster"]=test[col+"_final_cluster"].map(cat_labels2)

            correlated_features=correlated_features+[col+"_pca_comb_final",col+"_final_cluster"]

            # See which transformation along with the original is giving you the best univariate fit with target
            kf=KFold(n_splits=5, shuffle=True, random_state=42)

            rmse_scores = []

            for f in transformed_columns:
                X = train_copy[[f]].values
                y = train_copy[target].astype(int).values

                rmses = []
                for train_idx, val_idx in kf.split(X, y):
                    X_train, y_train = X[train_idx], y[train_idx]
                    x_val, y_val = X[val_idx], y[val_idx]
                    model=LinearRegression()
                    model.fit(X_train,np.log1p(y_train))
                    y_pred=nearest(np.expm1(model.predict(x_val)))
                    rmses.append(RMSE(np.log1p(y_val),np.log1p(y_pred)))
                    
                if f not in best_cols:
                    rmse_scores.append((f,np.mean(rmses)))
            if len(rmse_scores) > 0:
                best_col, best_rmse=sorted(rmse_scores, key=lambda x:x[1], reverse=False)[0]
                #print(f"{best_col}, {best_rmse}")
                best_cols.append(best_col)

                cols_to_drop = [f for f in correlated_features if  f not in best_cols]
                if cols_to_drop:
                    final_drop_list=final_drop_list+cols_to_drop
                table.add_row([col, best_col, best_rmse])

print(table)

In [ ]:
len(final_drop_list)

In [ ]:
len(train.columns)

In [ ]:
final_features = [f for f in train.columns if f not in final_drop_list]
tmp = [f for f in final_features if f not in "Rings"]
final_features = tmp
len(final_features)

In [ ]:
train[final_features].head()

In [ ]:
#train_scaled=train.copy()
#test_scaled=test.copy()

train = train[final_features]
test = test[final_features]

train_scaled = train
test_scaled = test

train_scaled=sc.fit_transform(train)
test_scaled=sc.transform(test)

In [ ]:
train = train_scaled
test = test_scaled

In [ ]:
train

## 4.5 Save data

Save prepared data to a file

In [ ]:
np.save('data/prepared_final_train.npy', train)
np.save('data/prepared_final_test.npy', test)
np.save('data/targets.npy', y)
#train.save('data/prepared_final_train.npy')
#test.save('data/prepared_final_test.npy')
#y.save('data/targets.npy')

# 5. Creating a model

In [25]:
# Reimport data
train = np.load('data/prepared_final_train.npy')
test = np.load('data/prepared_final_test.npy')
submission = pd.read_csv('data/sample_submission.csv')
y = np.load('data/targets.npy')

# Convert target to Pandas Series
#y = y.to_dict('dict')['Rings']
#y = pd.Series(data=y.values(), index=y.keys())

## 5.1 XGBoost Regression

### 5.1.1 Automatic hyperparameter tuning

Hyperparameter tuning with `optuna`

In [ ]:
%%time
def objective(trial, data=train, target=y, average=0):
    param = {
        'n_jobs': trial.suggest_categorical('n_jobs', [-1]),
        'n_estimators': trial.suggest_categorical('n_estimators', [400]),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise']),
        'eval_metric': trial.suggest_categorical('eval_metric', ['rmsle']),
        'objective': trial.suggest_categorical('objective', ['reg:squarederror']),
        'device' : trial.suggest_categorical('device', ['cuda']),
        'tree_method': trial.suggest_categorical('tree_method', ['hist']),  # this parameter means using the GPU when training our model to speedup the training process
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'gamma': trial.suggest_float('gamma', 0.0, 5),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'eta': trial.suggest_float('subsample', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'max_bin': trial.suggest_int('max_bin', 255, 8192),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_lambda': trial.suggest_loguniform('lambda', 1.0, 100.0),
        'reg_alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bylevel': trial.suggest_categorical('colsample_bylevel', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'random_state': trial.suggest_categorical('random_state', [42]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
    }

    # Count folds
    c = 1
    
    # Instantiate a cross-validator
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    # Fit the model with every cross-validation split
    for train_idx, val_idx in kf.split(data, target):
        X_train, y_train = data[train_idx], target[train_idx]
        x_val, y_val = data[val_idx], target[val_idx]
        xgbr_model = TransformedTargetRegressor(xgb.XGBRegressor(**param),
                                                func=np.log1p,
                                                inverse_func=np.expm1)
        xgbr_model.fit(X_train, y_train, eval_set=[(x_val,y_val)],verbose=0,callbacks=[EarlyStopping(rounds = 300,save_best=True)])
        y_preds = xgbr_model.predict(x_val)
        rmsle = RMSLE(y_val, y_preds)
        average += rmsle
        c += 1
    gc.collect()
    return average/(c-1)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

In [ ]:
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Mean RMSLE value:', study.best_trial.value)

### 5.1.2 Manual hyperparameter tuning

Manual fine-tuning of parameters set by `optuna`

In [ ]:
xgbr_params = study.best_trial.params
xgbr_params

In [39]:
xgbr_params = {
 'n_jobs': -1,
 'n_estimators': 776,
 'grow_policy': 'depthwise',
 'eval_metric': 'rmsle',
 'objective': 'reg:squarederror',
 'device': 'cpu',
 'tree_method': 'hist',
 'learning_rate': 0.014737232233402146,
 'gamma': 0.1706310268870559,
 'subsample': 0.3292036531972937,
 'max_depth': 9,
 'max_bin': 65535,
 'colsample_bytree': 0.6974727367761323,
 'lambda': 5.734442900265066,
 'alpha': 0.022404998725219973,
 'colsample_bylevel': 0.4,
 'random_state': 42,
 'min_child_weight': 3
}

In [12]:
%%time
def xgbr_cross_val(train, y, test, params, average=0):
    """
    Do a K-Fold data cross-validation on a trained XGBRegressor model
    """
    # Make a copy of an original training data set
    train_fold = train.copy()
    y_fold = y.copy()

    # Instantiate a numpy array filled with zeroes
    results = np.zeros(len(test), dtype=np.float32)
    
    # Instantiate a cross-validator
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    # Prepare data for cross-valdiation split
    X = train_fold
    y = y_fold

    # Instantiate PrettyTable
    table = PrettyTable()
    table.field_names = ['Fold', 'Train RMSLE', 'Test RMSLE', 'Average RMSLE']
    
    # Define best score (set to a high value as lower the score, the better)
    best_score = 1e5

    # Count folds
    c = 1

    # Fit the model with every cross-validation split
    for train_idx, val_idx in kf.split(X, y):
        print(f"{c} fold")
        X_train, y_train = X[train_idx], y[train_idx]
        x_val, y_val = X[val_idx], y[val_idx]
        xgbr_model = xgb.XGBRegressor(**params)
        xgbr_model.fit(X_train, y_train, eval_set=[(x_val,y_val)],verbose=2,callbacks=[EarlyStopping(rounds = 500,save_best=True)])
        y_preds = xgbr_model.predict(x_val)
        predictions = xgbr_model.predict(test)
        results = np.add(results, predictions)
        rmsle = RMSLE(y_val, y_preds)
        average += rmsle
        table.add_row([c, xgbr_model.best_score, rmsle, average/c])
        if xgbr_model.best_score < best_score:
            best_score = rmsle
        c += 1

    # Print table
    print(table)
    
    # Print RMSLE stats
    print(f"Best RMSLE: {best_score}, mean RMSLE: {average/(c-1)}")

    # Collect garbage
    gc.collect()
    return best_score, results/(c-1)

best_score, predictions = xgbr_cross_val(train, y, test, xgbr_params)

1 fold



KeyboardInterrupt



In [16]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.15)

In [38]:
%%time

# Instantiate a model
xgbr_model = xgb.XGBRegressor(**xgbr_params)

# Fit the model with training data
xgbr_model.fit(X_train, y_train, eval_set=[(X_test,y_test)],verbose=2,callbacks=[EarlyStopping(rounds = 300,save_best=True)])
predictions = xgbr_model.predict(test)

gc.collect()

[0]	validation_0-rmsle:0.28598
[2]	validation_0-rmsle:0.28054
[4]	validation_0-rmsle:0.27518
[6]	validation_0-rmsle:0.27006
[8]	validation_0-rmsle:0.26511
[10]	validation_0-rmsle:0.26044
[12]	validation_0-rmsle:0.25592
[14]	validation_0-rmsle:0.25153
[16]	validation_0-rmsle:0.24729
[18]	validation_0-rmsle:0.24323
[20]	validation_0-rmsle:0.23934
[22]	validation_0-rmsle:0.23555
[24]	validation_0-rmsle:0.23186
[26]	validation_0-rmsle:0.22837
[28]	validation_0-rmsle:0.22501
[30]	validation_0-rmsle:0.22172
[32]	validation_0-rmsle:0.21859
[34]	validation_0-rmsle:0.21553
[36]	validation_0-rmsle:0.21262
[38]	validation_0-rmsle:0.20982
[40]	validation_0-rmsle:0.20715
[42]	validation_0-rmsle:0.20457
[44]	validation_0-rmsle:0.20212
[46]	validation_0-rmsle:0.19972
[48]	validation_0-rmsle:0.19743
[50]	validation_0-rmsle:0.19523
[52]	validation_0-rmsle:0.19313
[54]	validation_0-rmsle:0.19110
[56]	validation_0-rmsle:0.18917
[58]	validation_0-rmsle:0.18731
[60]	validation_0-rmsle:0.18553
[62]	validati

3368

In [ ]:
# Create submission

submission["Rings"] = predictions
submission.to_csv('submission.csv', index=False)
submission.head()

## 5.2 CatBoost Regression

Tune hyperparameters and cross-validate Catboost regressor

### 5.2.1 Hyperparameter tuning

In [ ]:
%%time
def objective(trial, data=train, target=y, average=0):
    param = {
        'iterations': trial.suggest_categorical('iterations', [300]),
        "verbose": trial.suggest_categorical('verbose', [False]),
        'boosting_type': trial.suggest_categorical('boosting_type', ['Plain']),
        'depth': trial.suggest_int('depth', 6, 10),
        'max_bin': trial.suggest_int('max_bin', 128, 2048),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 100),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'random_strength': trial.suggest_float('random_strength', 1, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'max_leaves': trial.suggest_int('max_leaves', 2, 64),
        'eval_metric': trial.suggest_categorical('eval_metric', ['Quantile']),
        'loss_function': trial.suggest_categorical('loss_function', ['RMSE']),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bernoulli']),
        "grow_policy": trial.suggest_categorical('grow_policy', ['Lossguide']),
        "task_type": trial.suggest_categorical('task_type', ["GPU"]),
        'subsample': trial.suggest_float('subsample', 0.05, 1.0),
        #'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.05, 1.0),
        "random_state": trial.suggest_categorical('random_state', [42]),
        "early_stopping_rounds": trial.suggest_categorical('early_stopping_rounds', [300])
    }
    
    # Count folds
    c = 1
    
    # Instantiate a cross-validator
    kf = KFold(n_splits=2, shuffle=True, random_state=42)

    # Fit the model with every cross-validation split
    for train_idx, val_idx in kf.split(data, target):
        print(f"Fold {c}")
        X_train, y_train = data[train_idx], target[train_idx]
        x_val, y_val = data[val_idx], target[val_idx]
        # Get training and testing data
        train = Pool(data=X_train, label=y_train)
        evaluate = Pool(data=x_val, label=y_val)
        # Instantiate the model
        model = CatBoostRegressor(**param)
        # Fit the model
        model.fit(train, use_best_model=True, eval_set=evaluate)
        y_preds = model.predict(x_val)
        rmsle = RMSLE(y_val, y_preds)
        average += rmsle
        c += 1
    gc.collect()
    return average/(c-1)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1)
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best trial value:', study.best_trial.value)

[I 2024-04-29 08:35:51,939] A new study created in memory with name: no-name-24d6141d-2ee3-459f-855d-f3fa419cd805


Fold 1


Exception ignored in: <function Booster.__del__ at 0x74b15aa822a0>
Traceback (most recent call last):
  File "/home/kostas/Documents/git/Kaggle_Playground_Series/lib/python3.12/site-packages/xgboost/core.py", line 1753, in __del__
    _check_call(_LIB.XGBoosterFree(self.handle))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


KeyboardInterrupt: 

Exception ignored in: '_catboost._py_target_type_to_raw_target_data'
Traceback (most recent call last):
  File "/home/kostas/Documents/git/Kaggle_Playground_Series/lib/python3.12/site-packages/numpy/core/numerictypes.py", line 357, in issubdtype
    @set_module('numpy')
    
KeyboardInterrupt: 


In [ ]:
cat_params = study.best_trial.params
cat_params

In [34]:
cat_params = {
 'iterations': 267,
 'verbose': True,
 'depth': 10,
 'max_bin': 65535,
 'l2_leaf_reg': 31.80480657168747,
 'min_data_in_leaf': 20,
 'random_strength': 2.32234712463821,
 'learning_rate': 0.061882021628170504,
 'max_leaves': 57,
 'eval_metric': 'Quantile',
 'loss_function': 'RMSE',
 'bootstrap_type': 'Bernoulli',
 'grow_policy': 'Lossguide',
 'task_type': 'GPU',
 'subsample': 0.8982882178328477,
 'random_state': 42,
 'early_stopping_rounds': 300
}

### 5.2.2 Cross Validation

In [27]:
%%time

def cat_cross_val(train, y, test, param, average=0):
    """
    Do a K-Fold data cross-validation on a trained CatBoost Regressor model
    """
    # Make a copy of an original training data set
    X = train.copy()
    y = y.copy()
    
    # Instantiate a cross-validator
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    # Instantiate PrettyTable
    table = PrettyTable()
    table.field_names = ['Fold', 'Learn RMSE', 'Validation RMSE', 'Validation RMSLE', 'Mean RMSLE']
    
    # Define best score (set to a high value as lower the score, the better)
    best_score = 1e5

    # Count folds
    c = 1

    # Fit the model with every cross-validation split
    for train_idx, val_idx in kf.split(X, y):
        print(f"{c} fold")
        X_train, y_train = X[train_idx], y[train_idx]
        x_val, y_val = X[val_idx], y[val_idx]
        # Get training and testing data
        train = Pool(data=X_train, label=y_train)
        evaluate = Pool(data=x_val, label=y_val)
        cat_model = CatBoostRegressor(**param)
        cat_model.fit(train, use_best_model=True, eval_set=evaluate)
        y_preds = cat_model.predict(x_val)
        rmsle = RMSLE(y_val, y_preds)
        average += rmsle
        table.add_row([c, cat_model.get_best_score()['learn']['RMSE'], cat_model.get_best_score()['validation']['RMSE'], rmsle, average/c])
        c += 1
    print(table)
    print(f"Best RMSLE score: {best_score}, mean RMSLE score: {average/(c-1)}")
    return best_score

best_score = cat_cross_val(train, y, test, cat_params)

1 fold
0:	learn: 1.1141068	test: 1.1184707	best: 1.1184707 (0)	total: 1.03s	remaining: 51m 43s
1:	learn: 1.0678180	test: 1.0722913	best: 1.0722913 (1)	total: 2.04s	remaining: 50m 58s
2:	learn: 1.0261401	test: 1.0306396	best: 1.0306396 (2)	total: 3.04s	remaining: 50m 33s
3:	learn: 0.9880963	test: 0.9928428	best: 0.9928428 (3)	total: 4.03s	remaining: 50m 19s
4:	learn: 0.9534277	test: 0.9584324	best: 0.9584324 (4)	total: 5.01s	remaining: 50m 2s
5:	learn: 0.9216005	test: 0.9272393	best: 0.9272393 (5)	total: 6.02s	remaining: 50m 2s
6:	learn: 0.8929677	test: 0.8990237	best: 0.8990237 (6)	total: 7.01s	remaining: 49m 57s
7:	learn: 0.8665658	test: 0.8731252	best: 0.8731252 (7)	total: 8.01s	remaining: 49m 54s
8:	learn: 0.8423834	test: 0.8495432	best: 0.8495432 (8)	total: 8.98s	remaining: 49m 45s
9:	learn: 0.8202772	test: 0.8277080	best: 0.8277080 (9)	total: 9.97s	remaining: 49m 40s
10:	learn: 0.7999792	test: 0.8079959	best: 0.8079959 (10)	total: 11s	remaining: 49m 38s
11:	learn: 0.7813572	test: 

In [30]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.15)
X_val = X_test
y_val = y_test
# Pool data
train = Pool(data=X_train, label=y_train)
evaluate = Pool(data=X_test, label=y_test)

In [33]:
%%time

# Instantiate the model
cat_model = CatBoostRegressor(**cat_params)

# Fit the model
cat_model.fit(train, use_best_model=True, eval_set=evaluate)

# Evaluate the model
y_preds = cat_model.predict(X_val)
rmsle = RMSLE(y_val, y_preds)
print(f"RMSLE: {rmsle}")

# Make predictions
predictions = cat_model.predict(test)

0:	learn: 1.1137825	test: 1.1177599	best: 1.1177599 (0)	total: 899ms	remaining: 44m 55s
1:	learn: 1.0675955	test: 1.0712215	best: 1.0712215 (1)	total: 1.74s	remaining: 43m 25s
2:	learn: 1.0258330	test: 1.0293035	best: 1.0293035 (2)	total: 2.55s	remaining: 42m 27s
3:	learn: 0.9878848	test: 0.9911697	best: 0.9911697 (3)	total: 3.38s	remaining: 42m 12s
4:	learn: 0.9531876	test: 0.9564052	best: 0.9564052 (4)	total: 4.2s	remaining: 41m 56s
5:	learn: 0.9216316	test: 0.9248769	best: 0.9248769 (5)	total: 5.03s	remaining: 41m 48s
6:	learn: 0.8928208	test: 0.8960997	best: 0.8960997 (6)	total: 5.85s	remaining: 41m 42s
7:	learn: 0.8665426	test: 0.8698473	best: 0.8698473 (7)	total: 6.66s	remaining: 41m 29s
8:	learn: 0.8421523	test: 0.8456782	best: 0.8456782 (8)	total: 7.47s	remaining: 41m 21s
9:	learn: 0.8199503	test: 0.8237677	best: 0.8237677 (9)	total: 8.29s	remaining: 41m 18s
10:	learn: 0.7997026	test: 0.8037859	best: 0.8037859 (10)	total: 9.1s	remaining: 41m 14s
11:	learn: 0.7812400	test: 0.785

In [28]:
# Create submission

submission["Rings"] = predictions
submission.to_csv('submission.csv', index=False)
submission.head()

,id,Rings
0,90615,9.629911
1,90616,9.891570
2,90617,9.644171
3,90618,10.513810
4,90619,7.621931


In [15]:
print(f"Best RMSLE score with {cat_params['iterations']} iterations is: {best_score}")

NameError: name 'best_score' is not defined

## 5.3 LightGBM Regression

### 5.3.1 Hyperparameter Tuning

In [ ]:
%%time
def objective(trial, data=train, target=y, average=0):
    param = {
        'num_iterations': trial.suggest_categorical('num_iterations', [300]),
        'device': trial.suggest_categorical('device', ['gpu']),
        'n_jobs': trial.suggest_categorical('n_jobs', [-1]),
        'verbose': trial.suggest_categorical('verbose', [-1]),
        'max_depth': trial.suggest_int('max_depth', 6, 10),
        'num_leaves' : trial.suggest_int('num_leaves', 2, 1000),
        'subsample_freq': trial.suggest_int("subsample_freq", 1, 7),
        'random_state': trial.suggest_categorical('random_state', [42]),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1, 10.0),
        'subsample': trial.suggest_float('subsample', 0.05, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.05, 1.0),
        "min_child_weight": trial.suggest_float("min_child_weight", 1., 50.),
    }

    # Count folds
    c = 1
    
    # Instantiate a cross-validator
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    # Fit the model with every cross-validation split
    for train_idx, val_idx in kf.split(data, target):
        print(f"Fold {c}")
        X_train, y_train = data[train_idx], target[train_idx]
        x_val, y_val = data[val_idx], target[val_idx]
        lgbm_model = LGBMRegressor(**param)
        lgbm_model.fit(X_train, y_train, eval_set=[(x_val,y_val)],eval_names=["valid"],eval_metric=['MSLE'])
        y_preds = lgbm_model.predict(x_val)
        rmsle = RMSLE(y_val, y_preds)
        average += rmsle
        c += 1
    return average/(c-1)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best trial value:', study.best_trial.value)

In [ ]:
lgbm_params = study.best_trial.params
lgbm_params

In [45]:
lgbm_params = {
 'num_iterations': 855,
 #'early_stopping': 300,
 'device': 'gpu',
 'n_jobs': -1,
 'verbose': 1,
 'max_depth': 7,
 'num_leaves': 239,
 'subsample_freq': 7,
 'random_state': 42,
 'min_child_samples': 176,
 'reg_alpha': 0.013992152157392416,
 'reg_lambda': 2.515505626893547,
 'subsample': 0.9369891807768302,
 'learning_rate': 0.029000539629072005,
 'colsample_bytree': 0.4406989114710534,
 'min_child_weight': 39.887991868149506
}

### 5.3.2 Cross Validation

In [ ]:
%%time
def lgbm_cross_val(train, y, test, param, average=0):
    """
    Do a K-Fold data cross-validation on a trained LGBM Regression model
    """
    # Make a copy of an original training data set
    train_fold = train.copy()
    y_fold = y.copy()
    
    # Instantiate a numpy array filled with zeroes
    results = np.zeros(len(test), dtype=np.float32)

    # Instantiate a cross-validator
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    # Prepare data for cross-valdiation split
    X = train_fold
    y = y_fold

    # Instantiate PrettyTable
    table = PrettyTable()
    table.field_names = ['Fold', 'Validation RMSE', 'Validation RMSLE', 'Mean RMSLE']
    
    # Define best score (set to a high value as lower the score, the better)
    best_score = 1e5

    # Count folds
    c = 1

    # Fit the model with every cross-validation split
    for train_idx, val_idx in kf.split(X, y):
        print(f"{c} fold")
        X_train, y_train = X[train_idx], y[train_idx]
        x_val, y_val = X[val_idx], y[val_idx]
        lgbm_model = LGBMRegressor(**param)
        lgbm_model.fit(X_train, y_train, eval_set=[(x_val,y_val)],eval_names=["valid"],eval_metric=['MSLE'])
        y_preds = lgbm_model.predict(x_val)
        predictions = lgbm_model.predict(test)
        results = np.add(results, predictions)
        rmsle = RMSLE(y_val, y_preds)
        average += rmsle
        table.add_row([c, lgbm_model.best_score_['valid']['l2'], rmsle, average/c])
        if best_score > rmsle:
            best_score = rmsle
        c += 1
    print(table)
    print(f"Best RMSLE score: {best_score}")
    return best_score, results/(c-1)

best_score, predictions = lgbm_cross_val(train, y, test, lgbm_params)

In [44]:
lgbm_model = LGBMRegressor(**lgbm_params)
lgbm_model.fit(X_train, y_train, eval_set=[(X_test,y_test)],eval_names=["valid"],eval_metric=['MSLE'])
y_preds = lgbm_model.predict(x_val)
rmsle = RMSLE(y_val, y_preds)
print(f"RMSLE: {rmsle}")

[LightGBM] [Warning] early_stopping_round is set=300, early_stopping=300 will be ignored. Current value: early_stopping_round=300
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 10396
[LightGBM] [Info] Number of data points in the train set: 80573, number of used features: 579
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce 940MX, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 76 dense feature groups (5.84 MB) transferred to GPU in 0.008318 secs. 1 sparse feature groups
[LightGBM] [Warning] early_stopping_round is set=300, early_stopping=300 will be ignored. Current value: early_stopping_round=300
[LightGBM] [Info] Start training from score 9.705385
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 76 dense feature groups (5.47 MB) transferred to GPU in 0.007719 secs. 1 sparse feature groups
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 300 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

NameError: name 'x_val' is not defined

In [ ]:
# Create submission

submission["Rings"] = predictions
submission.to_csv('submission.csv', index=False)
submission.head()

## 5.4 Voting Regressor

### 5.4.1 Hyperparameters

In [ ]:
xgbr_params = {
 'n_jobs': -1,
 'n_estimators': 3000,
 'grow_policy': 'depthwise',
 'eval_metric': 'rmsle',
 'objective': 'reg:squarederror',
 'device': 'cuda',
 'tree_method': 'hist',
 'learning_rate': 0.04404749587959873,
 'gamma': 0.10364742524805404,
 'subsample': 0.9376708896501253,
 'max_depth': 9,
 'max_bin': 1655,
 'colsample_bytree': 0.7880464684349316,
 'lambda': 8.31564500887863,
 'alpha': 2.083946759558932,
 'colsample_bylevel': 1.0,
 'random_state': 42,
 'min_child_weight': 2
}

In [ ]:
cat_params = {
 'iterations': 3000,
 'verbose': True,
 'depth': 10,
 'max_bin': 1419,
 'l2_leaf_reg': 4.300237034450856,
 'min_data_in_leaf': 86,
 'random_strength': 2.0251226145225476,
 'learning_rate': 0.05582135516107398,
 'max_leaves': 63,
 'eval_metric': 'Quantile',
 'loss_function': 'RMSE',
 'bootstrap_type': 'Bernoulli',
 'grow_policy': 'Lossguide',
 'task_type': 'CPU',
 'subsample': 0.9094234167344891,
 'random_state': 42,
 'early_stopping_rounds': 300
}

In [ ]:
lgbm_params = {
 'num_iterations': 3000,
 'eval_metric': 'Quantile',
 'n_jobs': -1,
 'verbose': -1,
 'max_depth': 10,
 'num_leaves': 932,
 'subsample_freq': 3,
 'random_state': 42,
 'n_estimators': 300,
 'min_child_samples': 89,
 'reg_alpha': 2.844326093512898,
 'reg_lambda': 0.001222604907941908,
 'subsample': 0.8528870880815199,
 'learning_rate': 0.03170736097885979,
 'colsample_bytree': 0.4884624716768982,
 'min_child_weight': 46.817468629286644
}

### 5.4.2 Cross Validation

In [ ]:
%%time
def voting_cross_val(train, y, test, xgbr_params, lgbm_params, average=0):
    """
    Do a K-Fold data cross-validation on a Voting Regressor model
    """
    # Make a copy of an original training data set
    train_fold = train.copy()
    y_fold = y.copy()
    
    # Instantiate a numpy array filled with zeroes
    results = np.zeros(len(test), dtype=np.float32)

    # Instantiate a cross-validator
    kf = KFold(n_splits=10, shuffle=True, random_state=42)

    # Prepare data for cross-valdiation split
    X = train_fold
    y = y_fold

    # Instantiate PrettyTable
    table = PrettyTable()
    table.field_names = ['Fold', 'Validation RMSE', 'Validation RMSLE', 'Mean RMSLE']
    
    # Define best score (set to a high value as lower the score, the better)
    best_score = 1e5

    # Count folds
    c = 1

    # Fit the model with every cross-validation split
    for train_idx, val_idx in kf.split(X, y):
        print(f"{c} fold")
        X_train, y_train = X[train_idx], y[train_idx]
        x_val, y_val = X[val_idx], y[val_idx]
        # Instantiate models
        xgb_model = xgb.XGBRegressor(**xgbr_params)
        cat_model = CatBoostRegressor(**cat_params)
        lgbm_model = LGBMRegressor(**lgbm_params)
        # Instantiate voting model
        voting_model = VotingRegressor(estimators=[('XGB', xgb_model), ('CAT', cat_model), ('LGBM', lgbm_model)], n_jobs=-1)
        # Fit voting model
        voting_model.fit(X_train, y_train)
        y_preds = voting_model.predict(x_val)
        predictions = voting_model.predict(test)
        results = np.add(results, predictions)
        rmsle = RMSLE(y_val, y_preds)
        average += rmsle
        table.add_row([c, voting_model.score(x_val, y_val), rmsle, average/c])
        if best_score > rmsle:
            best_score = rmsle
        c += 1
    print(table)
    print(f"Best RMSLE score: {best_score}")
    return best_score, results/(c-1)

best_score, predictions = voting_cross_val(train, y, test, xgbr_params, lgbm_params)

In [ ]:
# Instantiate models
xgb_model = xgb.XGBRegressor(**xgbr_params)
cat_model = CatBoostRegressor(**cat_params)
lgbm_model = LGBMRegressor(**lgbm_params)
# Instantiate voting model
voting_model = VotingRegressor(estimators=[('XGB', xgb_model), ('CAT', cat_model), ('LGBM', lgbm_model)], n_jobs=-1)
# Fit voting model
voting_model.fit(X_train, y_train)

/home/kostas/Documents/git/Kaggle_Playground_Series/lib/python3.12/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 10396
[LightGBM] [Info] Number of data points in the train set: 80573, number of used features: 579
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce 940MX, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 76 dense feature groups (5.84 MB) transferred to GPU in 0.033822 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 9.705385
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 76 dense feature groups (5.47 MB) transferred to GPU in 0.022077 secs. 1 sparse feature groups
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

In [ ]:
# Create submission

submission["Rings"] = predictions
submission.to_csv('submission.csv', index=False)
submission.head()

In [ ]:
best_score